# Code 

In [18]:
from pathlib import Path
from datetime import datetime
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio
from rasterio.warp import reproject, Resampling
from rasterio.features import shapes as rio_shapes
from shapely.geometry import shape as shp_shape
import matplotlib.pyplot as plt

# --------------------- RUTAS ---------------------
ROOT     = Path('C:/Users/cespe/OneDrive_J/OneDrive/Ejercicio 7/Documents/Documents/Maestria/Paper_micro_elementos/GitHub/Change_detection_AM')

# Años / imágenes
DIR_2021 = ROOT / "Data" / "Images" / "Elalab" / "2021"
DIR_2022 = ROOT / "Data" / "Images" / "Elalab" / "2022"
DIR_2024 = ROOT / "Data" / "Images" / "Elalab" / "2024"

# Muestreos
SHP_SAL  = ROOT / "Data" / "Shapefiles" / "Elalab_salt_UTM.shp"  # columna CE

# Resultados y figuras
RES_2021 = ROOT / "Results" / "Elalab" / "2021"
RES_2022 = ROOT / "Results" / "Elalab" / "2022"
RES_2024 = ROOT / "Results" / "elalab" / "2024"
RES_CONS = ROOT / "Results" / "elalab"

FIG_2021 = ROOT / "Images" / "Elalab" / "violins_2021"
FIG_2022 = ROOT / "Images" / "Elalab" / "violins_2022"
FIG_2024 = ROOT / "Images" / "Elalab" / "violins_2024"
FIG_FINAL= ROOT / "Images" / "Elalab" / "violins_final"

for d in [RES_2021, RES_2022, RES_2024, RES_CONS, FIG_2021, FIG_2022, FIG_2024, FIG_FINAL]:
    d.mkdir(parents=True, exist_ok=True)

# ------------------- PARÁMETROS -------------------
NDVI_THRESHOLDS = np.arange(0.15, 0.51, 0.01)   # umbrales a barrer
DELTA_THRESHOLD = 0.15                          # delta NDVI mínimo entre fechas
ID_COL  = "id"
SAL_COL = "CE"

# >>>>>> Tú eliges ESTE valor tras ver los violines por año <<<<<<
CHOSEN_THR = 0.33   # p.ej. 0.31

# ------------------ Funciones -------------------
def load_ndvi_stack_s2_2bands(folder, epsilon=1e-6):
    """
    Carga un stack NDVI a partir de imágenes Sentinel-2 de 2 bandas (B4=Red, B8=NIR).

    Parameters
    ----------
    folder : str or pathlib.Path
        Carpeta con archivos TIF/TIFF nombrados como `YYYY_MM_DD.tif`.
        Cada archivo debe tener exactamente 2 bandas (B4, B8) en ese orden.
    epsilon : float, optional
        Pequeño término numérico en el denominador del NDVI para estabilidad.

    Returns
    -------
    ndvi_stack : numpy.ndarray
        Arreglo de forma (T, H, W) con NDVI en float32, ordenado cronológicamente.
    dates : list of datetime.datetime
        Lista de fechas (longitud T) derivadas del nombre del archivo y ordenadas.
    profile : dict
        Perfil raster del primer archivo (útil para escritura/geo-referenciación).
    transform : affine.Affine
        Transformación espacial del primer archivo.

    Raises
    ------
    ValueError
        Si no hay TIF/TIFF en la carpeta o si algún archivo no tiene 2 bandas.

    Notes
    -----
    El NDVI se calcula como (NIR - Red) / (NIR + Red + epsilon).
    """
    files = [f for f in Path(folder).iterdir() if f.suffix.lower() in (".tif", ".tiff")]
    if not files:
        raise ValueError(f"No hay TIFs en {folder}")
    dates = [datetime.strptime(f.stem, "%Y_%m_%d") for f in files]
    files = [f for _, f in sorted(zip(dates, files))]
    ndvis = []
    profile = None
    transform = None
    for f in files:
        with rasterio.open(f) as src:
            arr = src.read()
            if arr.shape[0] != 2:
                raise ValueError(f"{f.name}: se esperaban 2 bandas; tiene {arr.shape[0]}")
            red = arr[0].astype(np.float32)  # B4
            nir = arr[1].astype(np.float32)  # B8
            ndvi = (nir - red) / (nir + red + epsilon)
            ndvis.append(ndvi)
            if profile is None:
                profile = src.profile
                transform = src.transform
    return np.stack(ndvis), dates, profile, transform


def load_ndvi_stack_planet_4bands(folder, epsilon=1e-6):
    """
    Carga un stack NDVI a partir de imágenes Planet de ≥4 bandas (3=Red, 4=NIR).

    Parameters
    ----------
    folder : str or pathlib.Path
        Carpeta con archivos TIF/TIFF nombrados `YYYY_MM_DD.tif`.
        Cada archivo debe tener al menos 4 bandas.
    epsilon : float, optional
        Pequeño término numérico en el denominador del NDVI para estabilidad.

    Returns
    -------
    ndvi_stack : numpy.ndarray
        Arreglo de forma (T, H, W) con NDVI en float32, ordenado cronológicamente.
    dates : list of datetime.datetime
        Fechas (longitud T) extraídas del nombre del archivo y ordenadas.
    profile : dict
        Perfil raster del primer archivo.
    transform : affine.Affine
        Transformación espacial del primer archivo.

    Raises
    ------
    ValueError
        Si no hay TIF/TIFF en la carpeta o si algún archivo tiene < 4 bandas.

    Notes
    -----
    El NDVI se calcula con Red=banda 3 (índice 2) y NIR=banda 4 (índice 3).
    """
    files = [f for f in Path(folder).iterdir() if f.suffix.lower() in (".tif", ".tiff")]
    if not files:
        raise ValueError(f"No hay TIFs en {folder}")
    dates = [datetime.strptime(f.stem, "%Y_%m_%d") for f in files]
    files = [f for _, f in sorted(zip(dates, files))]
    ndvis = []
    profile = None
    transform = None
    for f in files:
        with rasterio.open(f) as src:
            arr = src.read()
            if arr.shape[0] < 4:
                raise ValueError(f"{f.name}: se esperaban ≥4 bandas; tiene {arr.shape[0]}")
            red = arr[2].astype(np.float32)  # band 3
            nir = arr[3].astype(np.float32)  # band 4
            ndvi = (nir - red) / (nir + red + epsilon)
            ndvis.append(ndvi)
            if profile is None:
                profile = src.profile
                transform = src.transform
    return np.stack(ndvis), dates, profile, transform


def detect_first_appearance(ndvi_stack, ndvi_threshold=0.3, delta_threshold=0.1):
    """
    Detecta la primera aparición de vegetación por píxel en un stack NDVI.

    Parameters
    ----------
    ndvi_stack : numpy.ndarray
        Stack NDVI de forma (T, H, W) en el que T >= 2.
    ndvi_threshold : float, optional
        Umbral mínimo de NDVI en la fecha t para considerar vegetación.
    delta_threshold : float, optional
        Incremento mínimo entre fechas consecutivas (NDVI_t - NDVI_{t-1}) para
        considerar aparición.

    Returns
    -------
    first : numpy.ndarray
        Arreglo (H, W) float32 con el índice temporal t (1..T-1) donde aparece
        por primera vez la vegetación; `np.nan` si nunca aparece.

    Notes
    -----
    Un píxel se marca en el primer t que cumple:
    (NDVI_t - NDVI_{t-1}) > delta_threshold y NDVI_t > ndvi_threshold.
    """
    first = np.full(ndvi_stack.shape[1:], np.nan, dtype=np.float32)
    for t in range(1, ndvi_stack.shape[0]):
        prev = ndvi_stack[t - 1]
        curr = ndvi_stack[t]
        growth = (curr - prev > delta_threshold) & (curr > ndvi_threshold)
        upd = growth & np.isnan(first)
        first[upd] = t
    return first


def early_binary(ndvi_stack, thr, delta_thr):
    """
    Convierte la detección de primera aparición en máscara binaria anual.

    Parameters
    ----------
    ndvi_stack : numpy.ndarray
        Stack NDVI (T, H, W).
    thr : float
        Umbral NDVI a aplicar (NDVI_t > thr).
    delta_thr : float
        Incremento mínimo entre fechas consecutivas (ΔNDVI > delta_thr).

    Returns
    -------
    mask : numpy.ndarray
        Arreglo binario (H, W) uint8, 1 si el píxel mostró aparición temprana
        al menos una vez en el año, 0 en caso contrario.
    """
    return (~np.isnan(detect_first_appearance(ndvi_stack, thr, delta_thr))).astype(np.uint8)


def save_binary_raster(arr2d, profile, transform, out_tif):
    """
    Guarda una máscara binaria 2D como GeoTIFF de una sola banda (uint8).

    Parameters
    ----------
    arr2d : numpy.ndarray
        Máscara (H, W) o (1, H, W). Cualquier valor >0 se fuerza a 1 (uint8).
    profile : dict
        Perfil raster base (se adaptan height, width, count, dtype y transform).
    transform : affine.Affine
        Transformación espacial a escribir en el GeoTIFF.
    out_tif : str or pathlib.Path
        Ruta de salida del archivo GeoTIFF.

    Returns
    -------
    None

    Raises
    ------
    ValueError
        Si `arr2d` no es 2D ni (1, H, W).
    """
    a = np.asarray(arr2d)
    if a.ndim == 3 and a.shape[0] == 1:
        a = a[0]
    if a.ndim != 2:
        raise ValueError(f"Esperado 2D o (1,H,W), recibido {a.shape}")
    a = (a > 0).astype("uint8")
    meta = profile.copy()
    meta.update(driver="GTiff", count=1, dtype="uint8",
                height=a.shape[0], width=a.shape[1], transform=transform)
    Path(out_tif).parent.mkdir(parents=True, exist_ok=True)
    with rasterio.open(out_tif, "w", **meta) as dst:
        dst.write(a, 1)


def violin_plot(in_vals, out_vals, title, save_path, ylabel="CE"):
    """
    Genera un violin plot CE (IN vs OUT) y lo guarda como PNG.

    Parameters
    ----------
    in_vals : array-like
        Valores CE para puntos dentro de la máscara (IN).
    out_vals : array-like
        Valores CE para puntos fuera de la máscara (OUT).
    title : str
        Título del gráfico.
    save_path : str or pathlib.Path
        Ruta de salida del PNG.
    ylabel : str, optional
        Etiqueta del eje Y (por defecto "CE").

    Returns
    -------
    None

    Notes
    -----
    Aplica limpieza básica (descarta NaN/Inf), añade puntos dispersos (jitter)
    y guarda la figura con DPI=200.
    """
    in_vals = np.asarray(in_vals)
    in_vals = in_vals[np.isfinite(in_vals)]
    out_vals = np.asarray(out_vals)
    out_vals = out_vals[np.isfinite(out_vals)]
    if in_vals.size == 0 and out_vals.size == 0:
        print(f"[WARN] Sin datos para violín → {save_path}")
        return
    data, positions, labels = [], [], []
    if in_vals.size > 0:
        data.append(in_vals)
        positions.append(1)
        labels.append("AM")
    if out_vals.size > 0:
        data.append(out_vals)
        positions.append(2 if 1 in positions else 1)
        labels.append("TM")
    fig, ax = plt.subplots(figsize=(7, 5))
    ax.violinplot(data, positions=positions, showmeans=True, showextrema=True, showmedians=True)
    ax.set_xticks(positions)
    ax.set_xticklabels(labels)
    ax.set_ylabel(ylabel)
    ax.set_title(title)
    rng = np.random.default_rng(42)
    for vals, xpos in zip(data, positions):
        x = xpos + (rng.random(vals.size) - 0.5) * 0.25
        ax.plot(x, vals, 'o', alpha=0.45, markersize=3)
    ax.grid(True, alpha=0.2, linestyle=":")
    fig.tight_layout()
    Path(save_path).parent.mkdir(parents=True, exist_ok=True)
    fig.savefig(save_path, dpi=200)
    plt.close(fig)
    print(f"[VIOLIN] {save_path}")


def sample_points_vs_binary(pts_gdf, bin_arr, transform, id_col=ID_COL):
    """
    Separa valores CE de puntos en IN vs OUT según una máscara binaria.

    Parameters
    ----------
    pts_gdf : geopandas.GeoDataFrame
        Puntos de muestreo con columna 'geometry' y campo CE (global `SAL_COL`).
        Debe estar en el mismo CRS que el raster (`transform`).
    bin_arr : numpy.ndarray
        Máscara binaria (H, W) con 1=IN, 0=OUT.
    transform : affine.Affine
        Transformación espacial del raster (para convertir XY a filas/columnas).
    id_col : str, optional
        Nombre de la columna de identificador (no se usa para el cálculo).

    Returns
    -------
    in_vals : numpy.ndarray
        CE de puntos ubicados dentro (IN) de la máscara.
    out_vals : numpy.ndarray
        CE de puntos ubicados fuera (OUT) de la máscara.

    Notes
    -----
    La clasificación IN/OUT se realiza por localización puntual del píxel.
    """
    H, W = bin_arr.shape
    in_vals, out_vals = [], []
    inv = ~transform
    for _, row in pts_gdf.iterrows():
        c, r = inv * (row.geometry.x, row.geometry.y)
        r = int(round(r))
        c = int(round(c))
        if 0 <= r < H and 0 <= c < W:
            (in_vals if bin_arr[r, c] == 1 else out_vals).append(row[SAL_COL])
    return np.array(in_vals, dtype=float), np.array(out_vals, dtype=float)


def align_to_ref(bin_arr, src_profile, ref_profile, ref_transform):
    """
    Reproyecta/alinea una máscara binaria al grid de referencia (nearest).

    Parameters
    ----------
    bin_arr : numpy.ndarray
        Máscara origen (H_src, W_src) uint8 (0/1).
    src_profile : dict
        Perfil del raster origen (usa 'transform' y 'crs').
    ref_profile : dict
        Perfil del raster de referencia (usa 'height', 'width', 'crs').
    ref_transform : affine.Affine
        Transformación del raster de referencia.

    Returns
    -------
    aligned : numpy.ndarray
        Máscara (H_ref, W_ref) uint8 alineada al grid de referencia.

    Notes
    -----
    Usa `rasterio.warp.reproject` con `Resampling.nearest`.
    """
    out = np.zeros((ref_profile["height"], ref_profile["width"]), dtype=np.uint8)
    reproject(
        source=bin_arr.astype(np.uint8),
        destination=out,
        src_transform=src_profile["transform"], src_crs=src_profile["crs"],
        dst_transform=ref_transform,          dst_crs=ref_profile["crs"],
        resampling=Resampling.nearest
    )
    return (out > 0).astype(np.uint8)


def vectorize_and_save(mask2d, transform, crs, out_shp):
    """
    Vectoriza una máscara binaria (1=IN) y guarda el resultado como Shapefile.

    Parameters
    ----------
    mask2d : numpy.ndarray
        Máscara binaria (H, W) con valores {0,1}.
    transform : affine.Affine
        Transformación espacial asociada a la máscara.
    crs : dict or rasterio.crs.CRS
        Sistema de referencia espacial a asignar al GeoDataFrame.
    out_shp : str or pathlib.Path
        Ruta de salida del Shapefile.

    Returns
    -------
    None

    Notes
    -----
    Se vectorizan únicamente los píxeles con valor 1; si no hay geometrías se
    emite un aviso y no se escribe archivo.
    """
    mask_arr = mask2d == 1
    feats = [
        {"geometry": shp_shape(geom), "value": int(val)}
        for geom, val in rio_shapes(mask2d.astype("uint8"), mask=mask_arr, transform=transform)
        if val == 1
    ]
    if not feats:
        print("[WARN] No hay polígonos en el consenso (mask vacía).")
        return
    gdf = gpd.GeoDataFrame(feats, geometry="geometry", crs=crs)
    gdf.to_file(out_shp)


def sweep_year(year_label, year_dir, loader_fn, fig_dir, res_dir):
    """
    Barrido de umbrales por año: guarda rasters binarios y violines por umbral.

    Parameters
    ----------
    year_label : str
        Etiqueta del año (ej., 'cafine_2019').
    year_dir : str or pathlib.Path
        Carpeta con las imágenes del año.
    loader_fn : callable
        Función de carga NDVI para el sensor del año (S2 o Planet).
        Debe retornar (ndvi_stack, dates, profile, transform).
    fig_dir : str or pathlib.Path
        Carpeta de salida para PNG de violines.
    res_dir : str or pathlib.Path
        Carpeta de salida para rasters por umbral.

    Returns
    -------
    info : dict
        Diccionario con llaves:
        - 'profile' : dict
        - 'transform' : affine.Affine
        - 'saved' : dict[float -> pathlib.Path]  (umbral -> ruta TIF)
        - 'ndvi_stack' : numpy.ndarray

    Notes
    -----
    Usa los umbrales globales `NDVI_THRESHOLDS` y `DELTA_THRESHOLD`, y cruza
    con los puntos `SHP_SAL` (CE) para generar violines IN vs OUT.
    """
    ndvi_stack, dates, profile, transform = loader_fn(year_dir)
    pts = gpd.read_file(SHP_SAL)
    if SAL_COL not in pts.columns:
        raise ValueError(f"'{SAL_COL}' no está en {SHP_SAL}")
    if ID_COL not in pts.columns:
        pts[ID_COL] = np.arange(len(pts)) + 1
    if pts.crs != profile["crs"]:
        pts = pts.to_crs(profile["crs"])

    saved = {}
    for thr in NDVI_THRESHOLDS:
        bin_arr = early_binary(ndvi_stack, thr, DELTA_THRESHOLD)
        out_tif = res_dir / f"{year_label}_earlyveg_thr_{thr:.2f}.tif"
        save_binary_raster(bin_arr, profile, transform, out_tif)
        saved[float(thr)] = out_tif

        in_vals, out_vals = sample_points_vs_binary(pts, bin_arr, transform, ID_COL)
        vio_png = fig_dir / f"{year_label}_violin_thr_{thr:.2f}.png"
        violin_plot(in_vals, out_vals, f"NDVI threshold={thr:.2f}", vio_png, ylabel=SAL_COL)

    return {"profile": profile, "transform": transform, "saved": saved, "ndvi_stack": ndvi_stack}


def apply_chosen_threshold(year_label, ndvi_stack, profile, transform, chosen_thr, out_dir):
    """
    Aplica el umbral elegido a un año y guarda el raster binario final.

    Parameters
    ----------
    year_label : str
        Etiqueta del año (ej., 'cafine_2019').
    ndvi_stack : numpy.ndarray
        Stack NDVI (T, H, W) del año.
    profile : dict
        Perfil raster base para escritura.
    transform : affine.Affine
        Transformación espacial del raster.
    chosen_thr : float
        Umbral NDVI seleccionado por el usuario.
    out_dir : str or pathlib.Path
        Carpeta de salida para el raster final.

    Returns
    -------
    out_final : pathlib.Path
        Ruta del GeoTIFF final (binario) para el año.
    """
    bin_arr = early_binary(ndvi_stack, chosen_thr, DELTA_THRESHOLD)
    out_final = out_dir / f"{year_label}_earlyveg_FINAL_thr_{chosen_thr:.2f}.tif"
    save_binary_raster(bin_arr, profile, transform, out_final)
    return out_final


def final_overlap(chosen_thr, ref_info, final_maps):
    """
    Interseca los 3 mapas finales (uno por año), guarda consenso y violín final.

    Parameters
    ----------
    chosen_thr : float
        Umbral NDVI elegido (solo para nombrar archivos de salida).
    ref_info : dict
        Diccionario de referencia (normalmente del año 2019) con:
        - 'profile' : dict
        - 'transform' : affine.Affine
    final_maps : dict
        Diccionario {año: ruta_tif_final} con los 3 rasters binarios finales.

    Returns
    -------
    None

    Side Effects
    ------------
    - Escribe:
      * Raster consenso: `associated_mangrove_consensus_thr_XX.tif`
      * Shapefile consenso: `associated_mangrove_consensus_thr_XX.shp`
      * Violín final: `violins_final_thr_XX.png`
    - Imprime rutas de salida.

    Notes
    -----
    Alinea cada raster al grid de referencia si es necesario (nearest) y
    realiza la intersección estricta (3/3) para el consenso.
    """
    ref_prof = ref_info["profile"]
    ref_tr = ref_info["transform"]
    aligned = []
    for year in sorted(final_maps.keys()):
        with rasterio.open(final_maps[year]) as src:
            arr = (src.read(1) > 0).astype(np.uint8)
            prof = src.profile
            tr = src.transform
        if (prof["height"], prof["width"], tr) == (ref_prof["height"], ref_prof["width"], ref_tr):
            aligned.append(arr)
        else:
            aligned.append(align_to_ref(arr, prof, ref_prof, ref_tr))
    stack = np.stack(aligned, axis=0)
    cons = (stack.sum(axis=0) == stack.shape[0]).astype(np.uint8)

    out_cons_tif = RES_CONS / f"associated_mangrove_consensus_thr_{chosen_thr:.2f}.tif"
    save_binary_raster(cons, ref_prof, ref_tr, out_cons_tif)
    out_cons_shp = RES_CONS / f"associated_mangrove_consensus_thr_{chosen_thr:.2f}.shp"
    vectorize_and_save(cons, ref_tr, ref_prof["crs"], out_cons_shp)

    pts = gpd.read_file(SHP_SAL)
    if pts.crs != ref_prof["crs"]:
        pts = pts.to_crs(ref_prof["crs"])
    in_vals, out_vals = sample_points_vs_binary(pts, cons, ref_tr, ID_COL)
    vio_final = FIG_FINAL / f"violins_final_thr_{chosen_thr:.2f}.png"
    violin_plot(in_vals, out_vals, f"Consenso (3/3) – NDVI thr={chosen_thr:.2f}", vio_final, ylabel=SAL_COL)

    print(f"[FINAL] Raster:    {out_cons_tif}")
    print(f"[FINAL] Shapefile: {out_cons_shp}")
    print(f"[FINAL] Violín:    {vio_final}")
    
    
def remove_islands(gdf, output_file, min_area=100):
    """
    Elimina islas pequeñas de un GeoDataFrame.

    Parameters
    ----------
    gdf : geopandas.GeoDataFrame
        GeoDataFrame con geometrías poligonales.
    min_area : float, optional
        Área mínima para conservar una geometría.

    Returns
    -------
    geopandas.GeoDataFrame
        GeoDataFrame sin islas pequeñas.
    """

    gdf = gdf.explode(index_parts=True)
    gdf = gdf[gdf.geometry.area >= min_area].reset_index(drop=True)
    gdf["value"] = 1
    gdf = gdf.dissolve(by='value', as_index=False)
    gdf.to_file(output_file, driver='ESRI Shapefile')

    return gdf[gdf.geometry.area >= min_area].reset_index(drop=True)


# ========================= MAIN =========================
def main():
    # 1) Barrido por año (guarda rasters por umbral + violines por umbral)
    y2021 = sweep_year("elalab_2021", DIR_2021, load_ndvi_stack_planet_4bands,  FIG_2021, RES_2021)
    y2022 = sweep_year("elalab_2022", DIR_2022, load_ndvi_stack_planet_4bands,  FIG_2022, RES_2022)
    y2024 = sweep_year("elalab_2024", DIR_2024, load_ndvi_stack_planet_4bands,  FIG_2024, RES_2024)
    print("\n[OK] Barrido por año completo. Revisa los violines y elige CHOSEN_THR arriba.\n")

    # 2) Aplicar el UMBRAL ELEGIDO por el usuario a CADA año → 3 rasters finales
    if CHOSEN_THR is None:
        print("[ATENCIÓN] Define CHOSEN_THR (ej: CHOSEN_THR = 0.31) y vuelve a ejecutar.")
        return
    thr = float(CHOSEN_THR)
    f2021 = apply_chosen_threshold("elalab_2021", y2021["ndvi_stack"], y2021["profile"], y2021["transform"], thr, RES_2021)
    f2022 = apply_chosen_threshold("elalab_2022", y2022["ndvi_stack"], y2022["profile"], y2022["transform"], thr, RES_2022)
    f2024 = apply_chosen_threshold("elalab_2024", y2024["ndvi_stack"], y2024["profile"], y2024["transform"], thr, RES_2024)

    # 3) Solapamiento 3/3 y violín final
    final_overlap(thr, y2021, {2021: f2021, 2022: f2022, 2024: f2024})

if __name__ == "__main__":
    main()

[VIOLIN] C:\Users\cespe\OneDrive_J\OneDrive\Ejercicio 7\Documents\Documents\Maestria\Paper_micro_elementos\GitHub\Change_detection_AM\Images\Elalab\violins_2021\elalab_2021_violin_thr_0.15.png
[VIOLIN] C:\Users\cespe\OneDrive_J\OneDrive\Ejercicio 7\Documents\Documents\Maestria\Paper_micro_elementos\GitHub\Change_detection_AM\Images\Elalab\violins_2021\elalab_2021_violin_thr_0.16.png
[VIOLIN] C:\Users\cespe\OneDrive_J\OneDrive\Ejercicio 7\Documents\Documents\Maestria\Paper_micro_elementos\GitHub\Change_detection_AM\Images\Elalab\violins_2021\elalab_2021_violin_thr_0.17.png
[VIOLIN] C:\Users\cespe\OneDrive_J\OneDrive\Ejercicio 7\Documents\Documents\Maestria\Paper_micro_elementos\GitHub\Change_detection_AM\Images\Elalab\violins_2021\elalab_2021_violin_thr_0.18.png
[VIOLIN] C:\Users\cespe\OneDrive_J\OneDrive\Ejercicio 7\Documents\Documents\Maestria\Paper_micro_elementos\GitHub\Change_detection_AM\Images\Elalab\violins_2021\elalab_2021_violin_thr_0.19.png
[VIOLIN] C:\Users\cespe\OneDrive_J\

In [19]:

from pathlib import Path
import geopandas as gpd
import pandas as pd
import folium
from IPython.display import display  # para mostrar el mapa en el notebook

# ----------------- RUTAS Y PARÁMETROS -----------------
ROOT = Path(r"C:/Users/cespe/OneDrive_J/OneDrive/Ejercicio 7/Documents/Documents/Maestria/Paper_micro_elementos/GitHub/Change_detection_AM")
CHOSEN_THR = 0.33  # <--- umbral elegido

SHP_AM = ROOT / "Results" / "Elalab" / f"associated_mangrove_consensus_thr_{CHOSEN_THR:.2f}.shp"
SHP_PTS = ROOT / "Data" / "Shapefiles" / "Elalab_salt_UTM.shp"  # Debe tener columna 'CE'

OUT_XLSX = ROOT / "Results" / "elalab" / f"AM_points_summary_thr_{CHOSEN_THR:.2f}.xlsx"

# Crea carpeta de salida si no existe
OUT_XLSX.parent.mkdir(parents=True, exist_ok=True)

# ----------------- LECTURA -----------------
am = gpd.read_file(SHP_AM)
pts = gpd.read_file(SHP_PTS)

# Chequeos básicos
if "CE" not in pts.columns:
    raise ValueError("La columna 'CE' no existe en el shapefile de puntos.")

# Asegurar CRS compatible para el join
if am.crs is None:
    raise ValueError("El shapefile de AM no tiene CRS definido.")
if pts.crs != am.crs:
    pts = pts.to_crs(am.crs)

# ----------------- CLASIFICACIÓN IN / OUT -----------------
# Unir espacialmente puntos a polígonos AM (IN = dentro). Para incluir borde, usa predicate='intersects'
pts_join = gpd.sjoin(pts, am[["geometry"]], how="left", predicate="within")
pts_join["in_am"] = pts_join.index_right.notna().astype(int)

# Columnas limpias para exportar
id_cols = [c for c in ("id", "ID", "Id") if c in pts_join.columns]
keep_cols = id_cols + [c for c in pts.columns if c not in id_cols + ["geometry"]] + ["in_am"]
df_all = pts_join[keep_cols].copy()
df_in  = df_all[df_all["in_am"] == 1].copy()
df_out = df_all[df_all["in_am"] == 0].copy()

# ----------------- EXCEL -----------------
with pd.ExcelWriter(OUT_XLSX, engine="xlsxwriter") as xw:
    df_all.to_excel(xw, index=False, sheet_name="ALL")
    df_in.to_excel(xw,  index=False, sheet_name="IN_AM")
    df_out.to_excel(xw, index=False, sheet_name="OUT_AM")

print(f"[OK] Excel guardado: {OUT_XLSX}")
print(f"Total puntos: {len(df_all)} | IN: {len(df_in)} | OUT: {len(df_out)}")
if len(df_in):
    print(f"CE (IN)  - mediana: {df_in['CE'].median():.3f}, n={len(df_in)}")
if len(df_out):
    print(f"CE (OUT) - mediana: {df_out['CE'].median():.3f}, n={len(df_out)}")

# ----------------- MAPA FOLIUM (MOSTRAR, NO GUARDAR) -----------------
# Reproyectar a WGS84 para web
am_wgs  = am.to_crs(4326)
pts_wgs = pts_join.to_crs(4326)

# Centro del mapa: centroide del polígono (o unión si hay múltiples)
am_union = am_wgs.unary_union
center_lat = am_union.centroid.y
center_lon = am_union.centroid.x

m = folium.Map(location=[center_lat, center_lon], zoom_start=13, control_scale=True, tiles="CartoDB positron")

# Capa polígono AM
folium.GeoJson(
    am_wgs.__geo_interface__,
    name=f"Associated Mangrove (thr={CHOSEN_THR:.2f})",
    style_function=lambda x: {"color": "#586f7e", "weight": 2, "fillColor": "#1f78b4", "fillOpacity": 0.25}
).add_to(m)

# Capas de puntos IN / OUT
fg_in  = folium.FeatureGroup(name="Puntos IN (dentro AM)", show=True)
fg_out = folium.FeatureGroup(name="Puntos OUT (fuera AM)", show=True)

def popup_html(row):
    cols = []
    for k in id_cols:
        if k in row: cols.append(f"<b>{k}:</b> {row[k]}")
    cols.append(f"<b>CE:</b> {row.get('CE', 'NA')}")
    cols.append(f"<b>IN_AM:</b> {int(row['in_am'])}")
    return "<br>".join(cols)

for _, r in pts_wgs.iterrows():
    color = "green" if r["in_am"] == 1 else "red"
    marker = folium.CircleMarker(
        location=[r.geometry.y, r.geometry.x],
        radius=4,
        color=color, fill=True, fill_opacity=0.85, weight=1
    )
    marker.add_child(folium.Popup(popup_html(r), max_width=260))
    (fg_in if r["in_am"] == 1 else fg_out).add_child(marker)

m.add_child(fg_in)
m.add_child(fg_out)
folium.LayerControl(collapsed=False).add_to(m)


[OK] Excel guardado: C:\Users\cespe\OneDrive_J\OneDrive\Ejercicio 7\Documents\Documents\Maestria\Paper_micro_elementos\GitHub\Change_detection_AM\Results\elalab\AM_points_summary_thr_0.33.xlsx
Total puntos: 99 | IN: 11 | OUT: 88
CE (IN)  - mediana: 2.355, n=11
CE (OUT) - mediana: 79.021, n=88


C:\Users\cespe\AppData\Local\Temp\ipykernel_17172\4208518824.py:64: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  am_union = am_wgs.unary_union


In [20]:
# Mostrar en el notebook:
from IPython.display import display  # para mostrar el mapa en el notebook
display(m)  # <- clave: lo muestra inline; no se guarda a archivo

In [23]:
final_shapefile = Path(r"C:/Users/cespe/OneDrive_J/OneDrive/Ejercicio 7/Documents/Documents/Maestria/Paper_micro_elementos/GitHub/Change_detection_AM/Results/Elalab/associated_mangrove_consensus_thr_0.33.shp")
outpath = Path(r"C:/Users/cespe/OneDrive_J/OneDrive/Ejercicio 7/Documents/Documents/Maestria/Paper_micro_elementos/GitHub/Change_detection_AM/Results/Elalab/")    

import geopandas as gpd

# Leer el shapefile
gdf = gpd.read_file(final_shapefile)

# Llamar la función con un GeoDataFrame, no con la ruta
cleaned_vector = remove_islands(gdf, Path(outpath) / 'dissolved_cleaned.shp', min_area=1000)


In [24]:
import folium
import geopandas as gpd

# --- Entrada: cleaned_vector puede ser un GeoDataFrame o una ruta a shapefile ---
gdf_in = cleaned_vector
if isinstance(cleaned_vector, (str, bytes)):
    gdf_in = gpd.read_file(cleaned_vector)
try:
    # También soporta pathlib.Path
    from pathlib import Path
    if isinstance(cleaned_vector, Path):
        gdf_in = gpd.read_file(cleaned_vector)
except Exception:
    pass

# Asegurar CRS WGS84 para web
gdf_wgs = gdf_in.to_crs(4326)

# Crear mapa y ajustarlo al shapefile
m = folium.Map(zoom_start=12, control_scale=True, tiles="CartoDB positron")
bounds = gdf_wgs.total_bounds  # [minx, miny, maxx, maxy]
m.fit_bounds([[bounds[1], bounds[0]], [bounds[3], bounds[2]]])

# Capa vectorial
folium.GeoJson(
    gdf_wgs.__geo_interface__,
    name="Cleaned vector",
    style_function=lambda x: {
        "color": "#0b7285",
        "weight": 2,
        "fillColor": "#0b7285",
        "fillOpacity": 0.15
    },
    highlight_function=lambda x: {"weight": 3, "fillOpacity": 0.25}
).add_to(m)

# Control de capas
folium.LayerControl(collapsed=False).add_to(m)

# Mostrar en la celda
m